# 감정 기반 + 개인화 하이브리드 음식 추천 (Colab)
이 노트북은 감정 분석 → 감정-음식 1차 매핑 → 개인화 점수화를 거쳐 최종 추천을 제공합니다.
실제 서비스 구조에 맞게 모듈화되어 있으며, 간단한 규칙 기반부터 시작해 모델/데이터 소스로 확장할 수 있습니다.

## 0. 환경 준비
Colab에서 실행 시 아래 패키지 설치 셀을 필요에 따라 사용하세요. (기본 데모는 추가 설치 없이 동작합니다)

In [ ]:
# 선택: 한국어 모델, Firestore 클라이언트를 쓰고 싶을 때 사용
# !pip install konlpy transformers torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu118
# !pip install google-cloud-firestore python-dotenv
# !pip install pandas numpy scikit-learn lightgbm

import datetime
import random
from typing import List, Dict, Any


## 1. 감정 분석 모델 (데모: 규칙/키워드 기반)
- 실제에선 KoBERT, KLUE RoBERTa, KoELECTRA 또는 임베딩+분류기를 붙이세요.
- 여기서는 데모를 위해 간단한 규칙 기반 감정 분류기를 제공합니다.

In [ ]:
EMOTIONS = ['stress', 'sad', 'tired', 'lonely', 'neutral']

def simple_emotion_classifier(text: str) -> Dict[str, Any]:
    t = text.lower()
    # 매우 단순한 키워드 기반 (데모용)
    if any(k in t for k in ['스트레스', 'stress', '짜증', '빡침']):
        return {'emotion': 'stress', 'score': 0.9}
    if any(k in t for k in ['슬프', 'sad', '우울', '눈물']):
        return {'emotion': 'sad', 'score': 0.85}
    if any(k in t for k in ['피곤', 'tired', '졸려', '지침']):
        return {'emotion': 'tired', 'score': 0.8}
    if any(k in t for k in ['외로', 'lonely', '쓸쓸', '허전']):
        return {'emotion': 'lonely', 'score': 0.8}
    return {'emotion': 'neutral', 'score': 0.6}

# 예시
simple_emotion_classifier('오늘 스트레스가 너무 쌓였어…')


## 2. 감정 → 음식 1차 매핑 (Emotion → Food Candidates)
- 실제 서비스에서는 Firestore 등에 저장/관리하는 것을 권장합니다.
- 데모에서는 파이썬 딕셔너리로 정의합니다.

In [ ]:
emotion_food_map: Dict[str, List[str]] = {
    'stress': ['짬뽕', '삼겹살', '떡볶이', '마라탕', '불고기', '카레'],
    'sad': ['미역국', '닭죽', '칼국수', '된장찌개', '솥밥'],
    'tired': ['닭가슴살 샐러드', '연어덮밥', '두부구이', '콩나물국밥', '비빔밥'],
    'lonely': ['치킨', '파스타', '피자', '햄버거', '크림리조또'],
    'neutral': ['김치찌개', '비빔국수', '순두부찌개', '샌드위치']
}

# 선택: 감정 점수가 높을수록 상위 일부만 후보로 제한하는 로직
def get_candidates_by_emotion(emotion: str, score: float, top_k: int = 5) -> List[str]:
    pool = emotion_food_map.get(emotion, emotion_food_map['neutral'])
    if score >= 0.8:
        return pool[:top_k]
    return pool

get_candidates_by_emotion('stress', 0.9)


## 3. 개인화 프로필 스키마 (Firestore/DB 연동 전 가상 데이터)
- 실제에선 `users/{uid}/profile`, `users/{uid}/preferences`, `users/{uid}/recent_food_logs` 구조로 저장 가능
- 데모에서는 파이썬 딕셔너리로 유저 프로필을 흉내냅니다.

In [ ]:
# 가상 유저 프로필
user_profile = {
    'uid': 'demo_user',
    'likes': ['삼겹살', '비빔밥', '파스타'],
    'dislikes': ['마라탕'],
    'recent': ['비빔밥'],  # 최근 24시간 내 섭취했다고 가정
    'sensitive_spicy': True,
    'health_flags': {
        'diabetes': False,
        'stomach_sensitive': True
    }
}

# 시간대/건강/음식 카테고리 예시
morning_foods = {'샌드위치', '닭가슴살 샐러드', '죽', '미역국'}
heavy_foods = {'삼겹살', '치킨', '피자'}
spicy_foods = {'짬뽕', '떡볶이', '마라탕', '김치찌개'}


## 4. 개인화 점수 함수
질문에서 제시한 가중치 공식을 그대로 반영했습니다.
- 감정 매칭 0.3
- 사용자 선호 0.3
- 시간대 적합 0.1
- 건강/체질 0.1 (데모에서는 매운맛 민감만 반영)
- 최근 섭취 패널티 0.2

In [ ]:
def personalized_score(food: str, user_profile: Dict[str, Any], emotion: str, emotion_score: float) -> float:
    score = 0.0

    # 1) 감정 점수 (감정 매핑 후보에 있을 때 가점)
    emotion_list = emotion_food_map.get(emotion, [])
    if food in emotion_list:
        score += 0.3

    # 2) 선호도 점수
    if food in user_profile.get('likes', []):
        score += 0.3
    if food in user_profile.get('dislikes', []):
        score -= 0.3

    # 3) 시간대 점수
    current_hour = datetime.datetime.now().hour
    if current_hour < 11 and food in morning_foods:
        score += 0.1
    if current_hour > 20 and food in heavy_foods:
        score -= 0.1

    # 4) 건강/체질 점수 (데모: 매운맛 민감)
    if user_profile.get('sensitive_spicy') and food in spicy_foods:
        score -= 0.1  # 건강 가중치 0.1 범주에 반영

    # 5) 최근 섭취 패널티
    if food in user_profile.get('recent', []):
        score -= 0.2

    return round(float(score), 4)


## 5. 최종 추천 함수

In [ ]:
def recommend_final(user_profile: Dict[str, Any], emotion: str, emotion_score: float, topn: int = 3):
    candidates = get_candidates_by_emotion(emotion, emotion_score, top_k=5)
    scored = []
    for food in candidates:
        s = personalized_score(food, user_profile, emotion, emotion_score)
        scored.append((food, s))
    scored.sort(key=lambda x: x[1], reverse=True)
    return scored[:topn]

# 데모 실행
emo = simple_emotion_classifier('오늘 스트레스가 정말 많이 쌓였고, 짜증나…')
final = recommend_final(user_profile, emo['emotion'], emo['score'], topn=3)
{
    'emotion': emo['emotion'],
    'top3': [ {'food': f, 'score': s} for f, s in final ]
}


## 6. 확장 포인트
- STT(음성→텍스트) 연동: Google STT 또는 Whisper API 사용 후 `simple_emotion_classifier` 입력으로 전달
- 한국어 대규모 언어모델 감정 분류: KLUE/KoBERT 파인튜닝 또는 임베딩+로지스틱 회귀
- Firestore 연동: `emotion_food_map`, `users/{uid}/...` 컬렉션/도큐먼트 구성 후 주기적 동기화
- 날씨/위치: OpenWeatherMap API로 현재 날씨 받아 점수 함수에 가중치 추가
- 반복 방지 로직 고도화: 최근 24~72시간/일주일 단위 히스토리 기반 패널티
- 모델 v2: LightGBM 회귀로 직접 `score` 예측(피처: 감정, 시간, 날씨, 유저 선호, 최근 로그 등)